In [10]:
# All modules that  are needed.
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline   

import rubin_sim.maf as maf
from rubin_sim.maf.maps import dustMap
#from rubin_sim.photUtils import Dust_values
#from rubin_sim.maf.metrics import crowdingMetric




from PulsatingStarRecovery_MAF import PulsatingStarRecovery

In [11]:
from rubin_sim.data import get_baseline

opsim_fname = get_baseline()
print(opsim_fname)

runName = os.path.split(opsim_fname)[-1].replace('.db', '')
print(runName)


#or take a new Opsim from the path (new http://astro-lsst-01.astro.washington.edu:8080/allMetricResults?runId=1)
#opsim_fname='/sims_maf/fbs_2.0/baseline/baseline_v2.0_10yrs.db'
#runName='baseline_v2.0_10yrs.db'




/sims_maf/rubin_sim_data/sim_baseline/baseline_v2.0_10yrs.db
baseline_v2.0_10yrs


In [12]:
# Connect to the database so we can use it with MAF.
opsim_db = maf.OpsimDatabase(opsim_fname)
#Finding available database columns with which to express constraints, slices, and metrics
#opsim_db.get_column_names("observations")


In [13]:
#From Jeff Carlin. This cell  builds a complex UserPointsSlicer
from astropy.io import ascii, fits
import healpy as hp
from astropy.coordinates import SkyCoord
import astropy.units as u


#Make a list of pointing.
#Read the Karachentsev+ catalog of nearby galaxies (new tale from Jeff d<9Mpc and dec < 45),
lv_dat0 = fits.getdata("lsst_galaxies_D_lt_9Mpc_table.fits")
# Keep only galaxies at d < 0.3 Mpc. and with M_B < -6.5.
lv_dat_cuts = (
        (lv_dat0["dist_Mpc"] < 0.035) & (lv_dat0["M_B"] > -6.5))
lv_dat = lv_dat0[lv_dat_cuts]

# Set up the slicer to evaluate the catalog we just made
slicer = maf.UserPointsSlicer(
        lv_dat["ra"], lv_dat["dec"], latLonDeg=True, badval=-666
    )
    # Add any additional information about each object to the slicer
slicer.slicePoints["distance"] = lv_dat["dist_Mpc"]

maps = ['DustMap']
print(slicer.slicePoints)
#dmod = 5*np.log10(slicer.slicePoints["distance"]*10**6)-5
#print(dmod)

{'sid': array([0, 1, 2, 3, 4, 5, 6, 7]), 'ra': array([0.58158734, 0.60766547, 0.65256406, 0.94117607, 2.00067092,
       2.64876985, 5.59167131, 6.26835001]), 'dec': array([ 0.63143103,  0.35212502, -1.37341898, -0.94333592, -0.97913456,
        0.28053744,  0.33365847, -1.04021623]), 'distance': array([0.03, 0.03, 0.03, 0.03, 0.03, 0.02, 0.02, 0.03])}


In [21]:
# Alternatively specify ra / dec of the point we want to work with on the sky - in degrees. 
# (these can be lists, if you want to work on multiple, limited points on the sky)
#test_ra =270.8839167 
test_ra =0.58158734
#test_dec =-30.0352667
test_dec =0.63143103
#dist_Mpc=0.01
dist_Mpc=0.03
#dmod=14.5


slicer = maf.UserPointsSlicer(test_ra, test_dec)
slicer.slicePoints["distance"] = dist_Mpc

maps = ['DustMap']



In [22]:
#This cell  build DataFrame df that contains  the  list of blending stars with magnitudes . Can be empty if you don't wont to consider blending or if you are offline.
#In that case comment from import to print

dati=[]
df=pd.DataFrame(dati,columns=[])

#import dl.queryClient as qc
#test_ra = test_ra
#test_dec = test_dec
#table = 'lsst_sim.simdr2'
#radius = 0.0002
#query = "select ra,dec,umag,gmag, rmag, imag, zmag, ymag, mu0, gc, mass from {} where q3c_radial_query(ra, dec, {}, {}, {})".format(table,test_ra,test_dec,radius)
#df = qc.query(sql=query ,timeout=1000, fmt='pandas') 
#print(df)

 

In [23]:
#parameters that must be inserted to run the metric PulsatingStarRecovery.py.Possible filenames are (https://github.com/MARCELLADC/PulsatingStarRecovery):
#filename='./RRab.csv'
#filename='./RRc.csv'
#filename='./CEF1.csv'
#filename='./CEF2.csv'
#filename='./LPV1.csv'
#filename='./LPV2.csv'
filename='./RRc.csv'   #ascii file with the template of pulsating star
#dmod=14.5    #distance modulus----> ORA L?HO MMESSO DOVE SCEGLIAMO RA E DEC
sigmaFORnoise=1   # sigma for the computation of the error on the magnitude
do_remove_saturated=True    #Remove from temporal series saurated visits  
factorForDimensionGap=0.5  #parameter used in LcSampling 
numberOfHarmonics=3         #parameter used in LcFitting--suggested value for a RRC





mymetric = PulsatingStarRecovery(filename,sigmaFORnoise,do_remove_saturated,numberOfHarmonics,factorForDimensionGap,df)
#if you are interested in other metrics
#maf.BaseMetric.help(doc=True)

./RRc.csv


In [24]:
# Define a sqlconstraint, if we need to just use a (large) subset of the opsim visits
#sqlconstraint = None   # no constraint, make all visits available

# Examples of other potentially useful sqlconstraints:
# sqlconstraint = 'filter = "r"'  # just select the visits in a particular filter
# sqlconstraint = 'note not like "%DD%"'  # don't choose any of the DD field visits
# sqlconstraint = 'night < 700'  # only use visits in the first year of the survey
sqlconstraint = 'night < 365'

In [25]:
bundle = maf.MetricBundle(mymetric, slicer, sqlconstraint,maps)

In [26]:
# Pass the bundle (along with any other bundles to be run on this opsim) to a MetricBundleGroup in order to
# calculate the metric bundle values.
g = maf.MetricBundleGroup({'PulsatingStarRecovery': bundle}, opsim_db, outDir='./', resultsDb=None)
# And calculate the metric
#from rubin_sim.maf.metrics import crowdingMetric
#import time
#start_time = time.time()
g.runAll()
#print("--- %s seconds ---" % (time.time() - start_time))

Querying database observations with constraint night < 365 for columns ['skyBrightness', 'filter', 'fieldRA', 'seeingFwhmEff', 'fiveSigmaDepth', 'visitExposureTime', 'observationStartMJD', 'fieldDec', 'rotSkyPos', 'airmass', 'night', 'numExposures']
Found 215980 visits
Running:  ['PulsatingStarRecovery']
0.03
sto analizzandola galassia con
0.02617397904396057
17.385606273598313
0.010150613970915719
801 801
Useful (at all S/N and NOT saturated)  Nvisits in ugrizy bands
3 7 13 17 17 21
Noise level (median vs mean)
0.12481687042708553 0.150757034079487
Finding optimal frequency:
 - Estimated peak width = 0.0178
 - Using 5 steps per peak; omega_step = 0.00355
 - User-specified period range:  0.038 to 0.73
 - Computing periods at 43719 steps
Zooming-in on 5 candidate peaks:
 - Computing periods at 995 steps
fitting...
fitting r band
fitting i band
fitting z band
fitting y band
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [20]:
bundle.metricValues.data[0]




{'n_u': 6,
 'n_g': 17,
 'n_r': 21,
 'n_i': 20,
 'n_z': 12,
 'n_y': 7,
 'maxGap_u': 0.317661869500526,
 'maxGap_g': 0.2957611530875166,
 'maxGap_r': 0.2802475344223865,
 'maxGap_i': 0.2146419526160921,
 'maxGap_z': 0.18515559347166288,
 'maxGap_y': 0.568672583296177,
 'numberGaps_u': 3,
 'numberGaps_g': 3,
 'numberGaps_r': 2,
 'numberGaps_i': 2,
 'numberGaps_z': 5,
 'numberGaps_y': 1,
 'uniformity_u': 0.9417709681491805,
 'uniformity_g': 0.8952268978971986,
 'uniformity_r': 999.0,
 'uniformity_i': 0.9407960732542342,
 'uniformity_z': 0.9489222901119487,
 'uniformity_y': 0.7572204330343884,
 'uniformityKS_u': 0.28837163011627825,
 'uniformityKS_g': 0.17827100420292735,
 'uniformityKS_r': 0.14655159569848375,
 'uniformityKS_i': 0.14728807482228062,
 'uniformityKS_z': 0.15223552446933775,
 'uniformityKS_y': 0.24527568298783614,
 'P_gatpsy': 0.38351231737975466,
 'Delta_Period': -1.5715091342682896e-06,
 'Delta_Period_abs': 0.00040976589891470263,
 'Delta_Period_abs_cicli': 9.65236386409223